In [ ]:
#### FOR ACADEMIC PROJECT WORK
#check if lib install or not, if not, install

# !pip install wfdb
# !pip install lightgbm
# !pip install PyWavelets
# !pip install biosppy
# !pip install torch
# !pip install shap

In [7]:
#keep updating progress and output as time goes by
import time
import os

def update_terminal():
    while True:
        # Clear jupyter notebook output
        os.system('clear')

        # Get the current time
        current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

        # Print the current time
        print(f"Current time: {current_time}")

        # Wait for 10 seconds before updating again
        time.sleep(30)

if __name__ == "__main__":
    update_terminal()



Current time: 2024-07-10 17:49:03
Current time: 2024-07-10 17:49:33
Current time: 2024-07-10 17:50:03
Current time: 2024-07-10 17:50:33
Current time: 2024-07-10 17:51:04
Current time: 2024-07-10 17:51:34
Current time: 2024-07-10 17:52:04
Current time: 2024-07-10 17:52:34
Current time: 2024-07-10 17:53:04
Current time: 2024-07-10 17:53:34
Current time: 2024-07-10 17:54:04
Current time: 2024-07-10 17:54:34
Current time: 2024-07-10 17:55:04
Current time: 2024-07-10 17:55:34
Current time: 2024-07-10 17:56:04
Current time: 2024-07-10 17:56:34
Current time: 2024-07-10 17:57:04
Current time: 2024-07-10 17:57:34
Current time: 2024-07-10 17:58:04
Current time: 2024-07-10 17:58:34
Current time: 2024-07-10 17:59:04
Current time: 2024-07-10 17:59:34
Current time: 2024-07-10 18:00:04
Current time: 2024-07-10 18:00:34
Current time: 2024-07-10 18:01:04
Current time: 2024-07-10 18:01:34
Current time: 2024-07-10 18:02:04
Current time: 2024-07-10 18:02:34
Current time: 2024-07-10 18:03:04
Current time: 

In [8]:

import os
#check number of files exist in each data folders
print('Number of files in each folder')
print('cpsc_processed:', len(os.listdir('/workspaces/ecg-arrhythmia-model/data/cpsc_processed')))
print('train_dataset:', len(os.listdir('/workspaces/ecg-arrhythmia-model/data/train_dataset')))
print('test_dataset:', len(os.listdir('/workspaces/ecg-arrhythmia-model/data/test_dataset')))

Number of files in each folder
cpsc_processed: 13757
train_dataset: 23320
test_dataset: 9248


In [ ]:
# from google.colab import drive
# drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# #set working directory to this folder
# import os...............
# os.chdir('/content/drive/Othercomputers/My Laptop/ecg-arrhythmia-model')

In [ ]:
import re
from datetime import datetime
import shutil
import os
from utils import load_dictionary_from_file
#load .hea file
file_name = 'data\ptb-xl\HR00001.hea'


In [ ]:
# DEBUG CODE - READ FILES

with open(file_name, 'r') as file:
    main_content = file.read()
# look for contains of the file between tag <code> and </code>
# and return the first match
# match = re.search(r'<code>(.*?)</code>', html_content, re.DOTALL)
if main_content:
    print(main_content)
else:
    print('No match')

In [ ]:
# DEBUG CODE - MODIFY FILES

#append date time to first line of match
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
# print(dt_string)
#split the match into lines
lines = main_content.split('\n')
#append date time to first line
lines[0] = lines[0] + ' ' + dt_string
#in the next 12 lines:

for i in range(1, 13):
    # lines[i] = lines[i][:12] + lines[i][14:] #remove 'x1' after 12th character
    # lines[i] = lines[i][:20] + lines[i][25:] #remove '.0(0)' after 20th character
    #get location of '.mat' in the line
    mat_loc = lines[i].find('.mat')+4 #location id at beginning of '.mat' -> +4 till the end
    # print(mat_loc)
    lines[i] = lines[i][:mat_loc+3] + lines[i][mat_loc+5:] #remove 'x1'
    lines[i] = lines[i][:mat_loc+11] + lines[i][mat_loc+16:] #remove '.0(0)'

#in the remaining lines, replace "#" with "# "
for i in range(13, len(lines)):
    lines[i] = lines[i].replace('# ', '#')


#join the lines back together
new_content = '\n'.join(lines)
print(new_content)

In [ ]:
#1. FUNCTION READ, AND MODIFY FILES
def process_file(file_name):
    with open(file_name, 'r') as file:
        main_content = file.read()
    # main_content = re.search(r'<code>(.*?)</code>', main_content, re.DOTALL) #no need to get part of the file
    file.close()
    if main_content:
        # lines = main_content.group(1).split('\n')
        lines = main_content.split('\n')
        now = datetime.now()
        dt_string = now.strftime("%d-%b-%Y %H:%M:%S")
        lines[0] = lines[0] + ' ' + dt_string #append date time to first line
        for i in range(1, 13):
            #get location of '.mat' in the line
            mat_loc = lines[i].find('.mat')+4 #location id at beginning of '.mat' -> +4 till the end
            # print(mat_loc)
            lines[i] = lines[i][:mat_loc+3] + lines[i][mat_loc+5:] #remove 'x1'
            lines[i] = lines[i][:mat_loc+11] + lines[i][mat_loc+16:] #remove '.0(0)'
        for i in range(13, len(lines)):
            lines[i] = lines[i].replace('# ', '#') #replace "#" with "# " in the remaining lines
        new_content = '\n'.join(lines)
        # new_file_name = file_name.replace('.hea', '_new.hea') #no need to change the file name
        #overwrite the original file
        with open(file_name, 'w') as file:
            file.write(new_content)
            file.close()
        # print('New file created:', file_name)
    else:
        print('No match')

In [ ]:
# CALL manual (read and modify files)
#loop through the files in main folder and process each file
files = os.listdir('data\CPSC_2018')
#make full path to each file
files = [os.path.join('data\CPSC_2018', file) for file in files]
for file in files:
    if file.endswith('.hea'):
        process_file(file)

In [ ]:
#2. FUNCTION MOVE FILES
def copy_files(folder, N):
    files = os.listdir(folder)
    #if N = 0, loop all files
    if N == 0:
        N = len(files)

    for i in range(N):
        file_name = os.path.join(folder, files[i])
        new_folder = os.path.dirname(folder) #copy to upper folder
        # new_folder = os.path.join(os.path.dirname(folder), 'main-data')
        shutil.copy(file_name, new_folder)
        # print('Copied:', file_name)
    print('Done, ', N, 'files copied in', folder)

# copy_files('.\data\CPSC_2018\g1', 50)

In [ ]:
#3. FUNCTION MOVE AND PROCESS FILES

def main_hea_process(data_folder, N):
    #make a list of sub-folders within the main data folder
    folders = os.listdir(data_folder)
    #loop through the folders and copy N files from each sub-folder
    for folder in folders:
        folder_name = os.path.join(data_folder, folder)
        if os.path.isdir(folder_name): #filter dirs only
            copy_files(folder_name, N)
    #loop through the files in main folder and process each file
    files = os.listdir(data_folder)
    #make full path to each file
    files = [os.path.join(data_folder, file) for file in files]
    for file in files:
        if file.endswith('.hea'):
            process_file(file)
    print('Done processing folders:', data_folder)


In [ ]:
#define main data folder string
data_folder = 'data\chapman_shaoxing'
# 'data\ptb-xl'
# 'data\cpsc_2018'
# 'data\cpsc_2018_extra'
# 'data\cpsc_processed'

#define samples get from each sub-folder
N = 0   #sampling files in each sub-folder.
        #Total file = N*number of sub-folder
        #N=0, get all files in each sub-folder

#process the main data folder
main_hea_process(data_folder, N)


In [ ]:
#Utility function to get unique codes from the files

def get_unique_dx_code (data_folder):
    files = os.listdir(data_folder)
    files = [os.path.join(data_folder, file) for file in files]
    list_codes = []
    for file in files:
        if file.endswith('.hea'):
            #read 16th line of the file
            with open(file, 'r') as f:
                lines = f.readlines()
                f.close()
            #get the 16th line
            line = lines[15]
            #get text after '#Dx: '
            text = line.split('#Dx: ')[1]
            #split the text into list of code, delimited by ','
            codes = text.split(',')
            #remove leading and trailing white spaces from each code
            codes = [code.strip() for code in codes]
            #append unique codes to the list
            for code in codes:
                if code not in list_codes:
                    list_codes.append(code)
    return list_codes

#call the function
# data_folder = 'data\cpsc_2018'
list_codes = get_unique_dx_code(data_folder)


In [ ]:
import json

# Function to load dictionary from file
def load_dictionary_from_file(filename):
    with open(filename, 'r') as file:
        dictionary = json.load(file)
    return dictionary

# Function to store dictionary to file
def store_dictionary_to_file(dictionary, filename):
    with open(filename, 'w') as file:
        json.dump(dictionary, file)

In [ ]:
# location of dx_dict file and class_labels file
file_dx_name = 'meta_data\dx_dict.json'  # File name to store the dictionary dx_dict
file_class_name = 'meta_data\class_labels.txt' # File name to store the class labels

# if dx_dict can be loaded from file, then load it, else use default dx_dict
if os.path.exists(file_dx_name):
    dx_dict = load_dictionary_from_file(file_dx_name)
else:
    #introduce dx_dict to store the codes
    dx_dict = {
            '426783006': 'SNR', # Normal sinus rhythm
            '164889003': 'AF', # Atrial fibrillation
            '270492004': 'IAVB', # First-degree atrioventricular block
            '164909002': 'LBBB', # Left bundle branch block
            '713427006': 'RBBB', # Complete right bundle branch block
            '59118001': 'RBBB', # Right bundle branch block
            '284470004': 'PAC', # Premature atrial contraction
            '63593006': 'PAC', # Supraventricular premature beats
            '164884008': 'PVC', # Ventricular ectopics
            '429622005': 'STD', # ST-segment depression
            '164931005': 'STE', # ST-segment elevation
        }


In [ ]:
#append the codes to the dictionary
for code in list_codes:
    if code not in dx_dict:
        # print('New code found:', code)
        #introduce new id sequence for new codes
        seq = 'Other' + str(len(dx_dict)-11+1) #original codes are 11
        dx_dict[code] = seq
# print(dx_dict)


In [ ]:
#check duplicate codes in dx_dict

#make a list of codes
codes = list(dx_dict.keys())
#make a list of unique codes
unique_codes = list(set(codes))
#check if the length of the two lists are the same
if len(codes) == len(unique_codes):
    print('No duplicate codes')
else:
    print('Duplicate codes found')
    #loop through the unique codes
    for code in unique_codes:
        #count the number of times the code appears in the list of codes
        count = codes.count(code)
        #if the count is greater than 1, print the code and the count
        if count > 1:
            print(code, count)

In [ ]:
# store new dx_dict to file
dx_dict = {
    '426783006': 'SNR', # Normal sinus rhythm
    '164889003': 'AF', # Atrial fibrillation
    '270492004': 'IAVB', # First-degree atrioventricular block
    '164909002': 'LBBB', # Left bundle branch block
    '713427006': 'RBBB', # Complete right bundle branch block
    '59118001': 'RBBB', # Right bundle branch block
    '284470004': 'PAC', # Premature atrial contraction
    '63593006': 'PAC', # Supraventricular premature beats
    '164884008': 'PVC', # Ventricular ectopics
    '429622005': 'STD', # ST-segment depression
    '164931005': 'STE', # ST-segment elevation
}
# filename = os.path.join(data_folder, filename)
store_dictionary_to_file(dx_dict, file_dx_name)

In [ ]:
#define function to make class labels from the dictionary

#function to get unique values from the dictionary -> as class labels
def get_unique_values(dictionary):
    values = list(dictionary.values())
    unique_values = list(set(values))
    return unique_values

#function to store unique values to file
def store_class_label_to_file(values, filename):
    with open(filename, 'w') as file:
        for value in values:
            file.write(value + '\n')
        file.close()

#function to load unique values from file and store in a list
def load_class_label_from_file(filename):
    class_label = []
    with open(filename, 'r') as file:
        for line in file:
            class_label.append(line.strip())
    return class_label

In [ ]:
file_class_name = 'meta_data\class_labels.txt' # File name to store the class labels
#get class labels from dictionary

class_labels = get_unique_values(dx_dict)
#store class labels to file
store_class_label_to_file(class_labels, file_class_name)

#load dict back to file
#dictionary = load_dictionary_from_file(filename)
#load class labels back to file
#class_labels = load_class_label_from_file(filename)

In [ ]:
#load dict back to file
dictionary = load_dictionary_from_file(filename)

In [ ]:
#Script to execute the above functions

# 'data\ptb-xl'
# 'data\cpsc_2018'
# 'data\cpsc_2018_extra'
# 'data\cpsc_processed'


#python preprocess.py --data-dir 'data\cpsc_2018_extra'
#python baselines.py --data-dir 'data\cpsc_2018_extra'  --classifier 'LR'
#python main.py --data-dir 'data\ptb-xl' --leads 'all' --epochs 2 --use-gpu --batch-size 200
#python predict.py --data-dir "data\cpsc_processed" --leads 'all' --use-gpu --batch-size 64

In [ ]:
#def function to move files .hea and .mat from one folder to another.
#Files name are extracted from 1st column of df
def move_files(df, source_folder, dest_folder):
    for index, row in df.iterrows():
        file_name = row['File']
        #append '.hea' and '.mat' to the file name, and move the files, one by one
        for ext in ['.hea', '.mat']:
            source_file = os.path.join(source_folder, file_name + ext)
            dest_file = os.path.join(dest_folder, file_name + ext)
            shutil.move(source_file, dest_file)
    print('Done moving files')

In [ ]:
# %run "preprocess.py" --data-dir "data/cpsc_processed" -- done in full
# %run "preprocess.py" --data-dir "data/train_dataset" -- done in full

#run function move file and handle .hea files
# %run "preprocess.py" --data-dir "data\chapman_shaoxing"
# %run "preprocess.py" --data-dir "data\test_dataset"
# %run "baselines.py" --data-dir "data\train_dataset" --classifier 'all'
# %run "main.py" --data-dir "data\cpsc_2018_extra" --leads 'all' --epochs 1 --batch-size 200

Number of keep records: 27823 Number of results: 27823 Percentage of keep records: 100.0


In [ ]:
%run "baselines.py" --data-dir "data/cpsc_processed" --classifier 'all'

data/cpsc_processed/labels.csv
Generating expert features...


100%|██████████| 6877/6877 [2:58:30<00:00,  1.56s/it]


Start training LR...
Finding optimal thresholds on validation dataset...
LR F1s: [0.27467811158798283, 0.8108108108108107, 0.8333333333333334, 0.5037037037037038, 0.4673913043478261, 0.6614785992217898, 0.35114503816793896, 0.3, 0.609053497942387]
Avg F1: 0.5346215999017525
Start training RF...
Finding optimal thresholds on validation dataset...
RF F1s: [0.3218390804597701, 0.8, 0.9142857142857143, 0.5283018867924528, 0.586046511627907, 0.6733333333333333, 0.3438914027149321, 0.3939393939393939, 0.6]
Avg F1: 0.573515258128167
Start training LGB...
[LightGBM] [Info] Number of positive: 481, number of negative: 5022
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.106001 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 160462
[LightGBM] [Info] Number of data points in the train set: 5503, number of used features: 636
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.087407 -> initscore=-2.345716
[LightG

In [ ]:
%run "baselines.py" --data-dir "data/train_dataset" --classifier 'all'

data/train_dataset/labels.csv
Generating expert features...


100%|██████████| 27823/27823 [5:12:40<00:00,  1.48it/s]


Start training LR...
Finding optimal thresholds on validation dataset...
LR F1s: [0.14342629482071714, 0.7941787941787942, 0.8414634146341463, 0.5059101654846335, 0.295, 0.5485519591141398, 0.22141560798548096, 0.29268292682926833, 0.9107049608355091]
Avg F1: 0.5059260137647432
Start training RF...
Finding optimal thresholds on validation dataset...
RF F1s: [0.17194570135746606, 0.8353413654618475, 0.8502994011976047, 0.46265060240963857, 0.38674033149171266, 0.6140089418777943, 0.2887139107611548, 0.4736842105263157, 0.9322381930184805]
Avg F1: 0.5572914064557793
Start training LGB...
[LightGBM] [Info] Number of positive: 1053, number of negative: 21204
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.234394 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 161751
[LightGBM] [Info] Number of data points in the train set: 22257, number of used features: 636
[LightGBM] [Info] [binary:BoostFromScore]: pa

In [ ]:
%run "main.py" --data-dir "data/cpsc_processed" --use-gpu

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/util

Training epoch 0:


  0%|          | 0/344 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEN

Loss: 110.4297
Validating...


100%|██████████| 43/43 [04:50<00:00,  6.75s/it]


...

Loss: 5.9456
F1s: [0.84466019 0.96747967 0.95714286 1.         0.9280397  0.73333333
 0.81118881 0.83636364 0.5625    ]
Avg F1: 0.8490
AUCs: [0.97804169 0.99002479 0.99851381 1.         0.98398728 0.9481203
 0.96521317 0.97701778 0.93244048]
Avg AUC: 0.9748
Training epoch 39:


  0%|          | 0/344 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 344/344 [02:13<00:00,  2.58it/s]


Loss: 3.2443
Validating...


100%|██████████| 43/43 [00:07<00:00,  5.42it/s]


Loss: 6.2399
F1s: [0.83937824 0.96666667 0.92       0.97777778 0.91707317 0.74782609
 0.81632653 0.85365854 0.5       ]
Avg F1: 0.8376
AUCs: [0.98064296 0.98914548 0.99677611 0.99993165 0.98086994 0.95157338
 0.96327322 0.97699824 0.92490079]
Avg AUC: 0.9738


In [ ]:
%run "main.py" --data-dir "data/train_dataset"
# --use-gpu

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/util

Training epoch 0:


  0%|          | 0/1392 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
  9%|▉         | 131/1392 [59:39<9:40:36, 27.63s/it]